In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
import gc
!pip install numpy pandas matplotlib scikit-learn tensorflow

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense



In [ ]:
# Paths
BASE_DIR = '../'
PATH_TO_TRAIN = os.path.join(BASE_DIR, "Dataset/train.parquet")
PATH_TO_MODELS = os.path.join(BASE_DIR, "LGBM/models")
#PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "LGBM/optuna/optuna_temp_artifacts")
#PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "LGBM/optuna/optuna_artifacts")


SEED = 42
BATCH_SIZE = 50
TEST_SIZE = 0.2
PATH_TO_TRAIN

In [ ]:
import optuna

def objective(trial):
    seq_len = trial.suggest_int("seq_len", 5, 60)

    X, y = create_sequences(data, seq_len)
    # split en train/test
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    model = Sequential([
        LSTM(50, input_shape=(seq_len, X_train.shape[2])),
        Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    loss = model.evaluate(X_test, y_test, verbose=0)

    return loss

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Mejor seq_len:", study.best_params["seq_len"])

In [ ]:
for product_id, group in productos:
    df = pd.read_parquet(PATH_TO_TRAIN)

    df = df[df['product_id'] == product_id][['periodo', 'clase']].copy()
    #df = df[df['product_id'] == product_id][['periodo', 'customer_id', 'clase']].copy()
    gc.collect()  # fuerza liberación de memoria no referenciada
    serie = group.sort_values('periodo')[['periodo', 'clase']]

    train, test = train_test_split(df,
                               test_size = TEST_SIZE,
                               random_state = SEED,
                               ) #stratify = dataset.clase saco esto porque no es un problema de clasificacion

    

In [ ]:
for product_id, group in df.groupby('product_id'):
    serie = group.sort_values('periodo')[['periodo', 'clase']]

    # Por ejemplo, podrías fijar un punto de corte para test/train
    split_index = int(len(serie) * (1-TEST_SIZE))
    train = serie.iloc[:split_index]
    test = serie.iloc[split_index:]

    # A modo ilustrativo, podrías aplicar un modelo sobre 'clase'
    # Por ejemplo, predicción naive para baseline
    pred = train['clase'].iloc[-1]
    test['pred_clase'] = pred

    print(f'Producto {product_id} - Última clase train: {pred}')